# Bulk Standard Geometry
A notebook to enrich **[standard geography](https://developers.arcgis.com/rest/geoenrichment/api-reference/standard-geography-query.htm) ids**, such as `census block groups`, with their geometries and publish as a geographic layer. This uses the [`standard_geography_query`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geoenrichment.html#standard-geography-query) function and there's a blog about these queries [here](https://blogs.esri.com/esri/arcgis/2017/07/10/best-practices-how-to-query-standard-geographies-branches/).

In [2]:
# common imports
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
from arcgis.geoenrichment import standard_geography_query

In [3]:
# import relative util function
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.cool_utils import chunk

## User Input
* **GIS**: Your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/). Not specifying a password creates a password prompt

In [ ]:
gis = GIS(username="mpayson_startups")

* **csv_path**: Path to the `csv` to be geocoded
* **lyr_title**: Title for output hosted layer
* **layers**: The standard geography layers whose geometry you want to fetch. Here's the [US list](http://geoenrich.arcgis.com/arcgis/rest/services/World/GeoenrichmentServer/Geoenrichment/StandardGeographyLevels/US/census?f=pjson) of layers.
* **f_std_id**: The `csv` field with the ids corresponding to a standard geography layer

In [4]:
csv_path = 'test_in.csv'
lyr_title = 'test_out.csv'
layers = ['US.BlockGroups']
f_std_id = 'census_block_group'

## Execution

In [ ]:
# read df
df = pd.read_csv(csv_path)

In [ ]:
# get and concat block geometries, chunking because of request time-outs
ids = [i for i in df[f_std_id]]
gdf = pd.concat((standard_geography_query(layers=layers, ids=c, return_geometry=True) for c in chunk(blocks)))

In [ ]:
# merge geometries
sdf = pd.merge(gdf, df, left_on='AreaID', right_on=f_std_id)

In [ ]:
# rebuild spatial dataframe, merge created unpredictable results when publishing
atr_df = sdf.drop('SHAPE', axis=1)
geom = [g for g in sdf['SHAPE']]
sdf2 = SpatialDataFrame(data=atr_df, geometry=geom)

In [ ]:
# Send it!
sdf2.to_featurelayer(lyr_title)